#### Import the relevant libraries

In [1]:
from PIL import Image
from PIL import ImageFilter
import numpy as np
import copy
import time
import math

#### Read and convert to greyscale

Open the image and convert it to greyscale since we are asked to convolve on greyscale images

In [2]:
def read_image(x):
    x = Image.open('test-images/' + x).convert("L")
    x = np.array(x)
    return x

In [3]:
pixels = read_image('music1.png')
pixels.shape

(231, 1274)

#### Custom kernels

In [4]:
def check_range(x): return (np.min(x), np.max(x))

In [5]:
check_range(pixels)

(0, 255)

**Some test kernels**

- a = identity
- b = box blur
- c = horizontal derivative
- d = gaussian
- e = sharpening
- f = derivative of gaussian

In [6]:
a = np.zeros((3,3))
a[1,1] = 1
a

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])

In [7]:
b = np.ones((3,3))
b

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [8]:
c = np.zeros((3,3))
c[1,0], c[1,2] = -1, 1
c

array([[ 0.,  0.,  0.],
       [-1.,  0.,  1.],
       [ 0.,  0.,  0.]])

In [9]:
d = np.array([[0.003,0.013,0.022,0.013,0.003],
             [0.013,0.059,0.097,0.059,0.013],
             [0.022,0.097,0.159,0.097,0.022],
             [0.013,0.059,0.097,0.059,0.013],
             [0.003,0.013,0.022,0.013,0.003]])
d

array([[0.003, 0.013, 0.022, 0.013, 0.003],
       [0.013, 0.059, 0.097, 0.059, 0.013],
       [0.022, 0.097, 0.159, 0.097, 0.022],
       [0.013, 0.059, 0.097, 0.059, 0.013],
       [0.003, 0.013, 0.022, 0.013, 0.003]])

In [10]:
# vary the value of alpha
e = np.zeros((5,5))
e[1:4,1:4] = a
alpha = 0.8 + 1
e = e * alpha - d
e

array([[-0.003, -0.013, -0.022, -0.013, -0.003],
       [-0.013, -0.059, -0.097, -0.059, -0.013],
       [-0.022, -0.097,  1.641, -0.097, -0.022],
       [-0.013, -0.059, -0.097, -0.059, -0.013],
       [-0.003, -0.013, -0.022, -0.013, -0.003]])

In [11]:
f = np.zeros((5,5))
f[1:4,1:4] = c
f = f * d
f

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.097,  0.   ,  0.097,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

#### Padding the image

In [12]:
def pad_image(x, kernel):
    """
    Create a new numpy array with all values 255 and then add the image in between
    i/p: image, kernel
    o/p: padded_kernelx
    """
    to_add = kernel.shape[0] - 1
    padded_image = np.full((x.shape[0] + to_add, x.shape[1] + to_add), 255)
    t = to_add // 2
    padded_image[t: padded_image.shape[0] - t, t: padded_image.shape[1] - t] = x
    return padded_image

In [13]:
p = pad_image(pixels, a)
p.shape

(233, 1276)

In [14]:
assert p.shape == (pixels.shape[0] + a.shape[0] - 1, pixels.shape[1] + a.shape[0] - 1)

In [15]:
def display_np(x):
    """
    Display a numpy array as an image
    """
    result = Image.fromarray(x.astype('uint8'))
    result.show()

In [16]:
display_np(pixels)

#### convolution

In [17]:
def conv2d(image, kernel, div = 1, clip = True):
    """
    Applies 2d convolution on a 2d image. Also works with rectangular kernels
    """

    # number of rows and columns of the kernel
    r = kernel.shape[0]
    c = kernel.shape[1]

    # initialize a canvas for the output with 255s. We will fill values in this
    output = np.full(image.shape, 255)
    for i in range(image.shape[0] - r - 1):
        for j in range(image.shape[1] - c - 1):
            output[i][j] = np.sum(kernel * image[i:i+r, j:j+c]) / div
    if clip: np.clip(output, 0, 255, out = output)
    return output
#     display_np(output)

In [18]:
# conv2d(p, a)
# conv2d(p, b, 9)
# conv2d(p, c)
# conv2d(p, d)
# conv2d(p, e)
# conv2d(p, f)

In [19]:
sobel = np.array([[-1, 0, 1],
                 [-2, 0, 2],
                 [-1, 0, 1]])

In [20]:
# conv2d(p, sobel)

#### Template matching

In [21]:
%ls test-images/

Invalid switch - "".


In [22]:
tmp1 = read_image('template1.png')
tmp2 = read_image('template2.png')
tmp3 = read_image('template3.png')

templates = [tmp1,tmp2,tmp3]

tmp1.shape, tmp2.shape, tmp3.shape

((11, 17), (35, 16), (29, 18))

In [23]:
canvas = np.full(p.shape, 255)

In [24]:
def find_threshold(y):
    """
    Takes a number y and returns a threshold eg 3542, thresh = 3000
    eg2 16258325 thresh 10000000
    """
    
    # to do: reduce lines of code
    str1, str2 = '', ''
    y = [int(i) for i in str(y)]
    length = len(y) - 1
    sub = y[-length:]

    for el_y in y:
        str1 += str(el_y)
    for el_y in sub:
        str2 += str(el_y)
    new_y = int(str1)
    sub_y = int(str2)
    thresh = new_y - sub_y
    return thresh

In [25]:
def temp_match(template, padded_im):
    """
    Takes a list of templates and an image and returns the matched template
    """
    idxs1, idxs2 = np.array([]), np.array([])
    
    # to do: pass a list of templates and work on that
    # also try the other images in the test_images
    # esp the big ranch image or something
    # write one line comments for what every bit of code is doing
    
    op = conv2d(padded_im, template, 1, False) + conv2d(255 - padded_im, 255 - template, 1, False)
    x,y = check_range(op)    
    thresh = find_threshold(y)
    temp_op = op[op > thresh]    
    for t in temp_op:
        t_idxs = np.where(op == t)
        idxs1 = np.append(idxs1 ,t_idxs[0])
        idxs2 = np.append(idxs2 ,t_idxs[1])
    for r,c in zip(idxs1, idxs2):
        r, c = int(r), int(c)
        if r+template.shape[0] < canvas.shape[0] and c+template.shape[1] < canvas.shape[1]:
            canvas[r:r+tmp3.shape[0], c:c+tmp3.shape[1]] = p[r:r+tmp3.shape[0], c:c+tmp3.shape[1]]


In [26]:
for template in templates:
    temp_match(template,p)

In [27]:
display_np(canvas)

In [49]:
# lines = p - canvas
# display_np(lines)

In [28]:
# op = conv2d(p, tmp3, 1, False) + conv2d(255 - p, 255 - tmp3, 1, False)
# op.shape

In [29]:
# check_range(op)

In [30]:
# temp_op = op[op > 30000000]

In [31]:
# # m = np.min(op)
# idxs1 = np.array([])
# idxs2 = np.array([])
# for t in temp_op:
#     t_idxs = np.where(op == t)
#     idxs1 = np.append(idxs1 ,t_idxs[0])
#     idxs2 = np.append(idxs2 ,t_idxs[1])

In [32]:
# len(idxs1)

In [33]:
# assert len(idxs1) == len(idxs1)

In [34]:
# check_range(idxs1), check_range(idxs2)

In [35]:
# canvas = np.full(p.shape, 255)

In [36]:
# check_range(canvas)

In [37]:
# for r,c in zip(idxs1, idxs2):
#     r, c = int(r), int(c)
#     if r+tmp3.shape[0] < canvas.shape[0] and c+tmp3.shape[1] < canvas.shape[1]:
# #         display_np(p[r:r+tmp1.shape[0], c:c+tmp1.shape[1]])
# #         break
#         canvas[r:r+tmp3.shape[0], c:c+tmp3.shape[1]] = p[r:r+tmp3.shape[0], c:c+tmp3.shape[1]]
# #         display_np(canvas)
# #         print(check_range(canvas))
# #         time.sleep(5)

In [38]:
# display_np(canvas)

#### 1d convolution

In [39]:
def conv1d(image, kernel, div = 1):
    """
    Performs 1d convolution on an image
    """
    
    im = copy.deepcopy(image.flatten())
    k = copy.deepcopy(kernel.flatten())

    size = len(k)
    to_remove = len(k) - 1
    
    output = np.full((im.shape), 255)
    for i in range(len(im) - to_remove):
        output[i] = np.sum(k * im[i:i+size]) / div
    return output

In [40]:
h1 = np.array([1,2,1])
h2 = np.array([-1,0,1])

In [41]:
tmp = conv1d(p, h1)

In [42]:
result = conv1d(tmp, h2)

In [43]:
check_range(result)

(-765, 765)

In [44]:
np.clip(result, 0, 255, out = result)
check_range(result)

(0, 255)

In [45]:
result = result.reshape(p.shape)
result.shape

(233, 1276)

In [46]:
# display_np(result)